## Frameshift finder
### Import fasta file
### Translate file need to use a single translation table. Need to group fasta input by tax groups if using alternative translation tables
### Create initial alignments of DNA and protein sequences
### Scan the DNA alignment for gaps < 3 bp if gap is found remove one nucleotide - translate - realign and see if this improves the protein alignment. If not try again removing two nucleotides. If either improves protein alignment score this as frameshift found. If not continue. Report any frameshift found and location

In [7]:
import Bio
import pandas as pd 
import os
import subprocess

### Read input file and translate

In [84]:
from Bio import SeqIO
coding_dna = []
protein_seq = []
trans = []
translations = []
for seq_record in SeqIO.parse("missingcytb.fsa", "fasta"):
    s = seq_record
    coding_dna = s.seq
    s.seq = s.seq.translate(table=5)
    protein_seq.append(s)
print(protein_seq)
SeqIO.write(protein_seq, "missingcytb.pro", "fasta")

[SeqRecord(seq=Seq('MTNPRKTYLLMKIINKTFIDLPTPSNISTWWNFGSLLGMCLMIQIITGLFLAMH...PK*'), id='NC_008133.1', name='NC_008133.1', description='NC_008133.1 Phascolarctos cinereus cytochrome b', dbxrefs=[]), SeqRecord(seq=Seq('MSHRLRLDNPLFKIINGTLIDLPSPVNFSIWWNFGSLLGLVLGIQLLTGIFLAM...IL*'), id='NC_013810.1', name='NC_013810.1', description='NC_013810.1 Zonosagitta nagae cytochrome b', dbxrefs=[]), SeqRecord(seq=Seq('MTMRFRLTNKLAKIVNGMVWDLGTPQNINMFWNFGSLLGLMLSIQLITGIFLAL...FG*'), id='NC_013811.1', name='NC_013811.1', description='NC_013811.1 Decipisagitta decipiens cytochrome b', dbxrefs=[]), SeqRecord(seq=Seq('MMNLYKPIRKTHPVIKIINGSLIDLPTPSNISSWWNFGSLLALCLMIQIITGLF...KS*'), id='NC_014587.1', name='NC_014587.1', description='NC_014587.1 Hipparchia autonoe cytochrome b', dbxrefs=[]), SeqRecord(seq=Seq('MSLSKTGLLEPVNDIIDLPTPSNISYWWNFGSLLCLCLGIQMMTGVLLAMHYSS...SS*'), id='NC_015309.1', name='NC_015309.1', description="NC_015309.1 Placozoan sp. 'Shirahama' cytochrome b", dbxrefs=[]), SeqRecord(seq=Seq('

34

### Create alignments

In [7]:
from Bio import AlignIO
help(AlignIO)

Help on package Bio.AlignIO in Bio:

NAME
    Bio.AlignIO - Multiple sequence alignment input/output as alignment objects.

DESCRIPTION
    The Bio.AlignIO interface is deliberately very similar to Bio.SeqIO, and in
    fact the two are connected internally.  Both modules use the same set of file
    format names (lower case strings).  From the user's perspective, you can read
    in a PHYLIP file containing one or more alignments using Bio.AlignIO, or you
    can read in the sequences within these alignments using Bio.SeqIO.
    
    Bio.AlignIO is also documented at http://biopython.org/wiki/AlignIO and by
    a whole chapter in our tutorial:
    
    * `HTML Tutorial`_
    * `PDF Tutorial`_
    
    .. _`HTML Tutorial`: http://biopython.org/DIST/docs/tutorial/Tutorial.html
    .. _`PDF Tutorial`: http://biopython.org/DIST/docs/tutorial/Tutorial.pdf
    
    Input
    -----
    For the typical special case when your file or handle contains one and only
    one alignment, use the func

In [105]:
from Bio import AlignIO

alignment = AlignIO.read(open("align.out"), "fasta")
print("Alignment length %i" % alignment.get_alignment_length())
for record in alignment:
    print(record.seq + " " + record.id)

Alignment length 1239
ATGATTAGG-------------------------------------------ATTATTCGACGAAATTTAGTGGATTTACCGACAAAATTTTCTTTAAGCTACTATTGGTGTAGCGGTTTTATGATTTCAGCATTTATGATTTTTCAAGTTGTCACGGGGGTTATACTGTCTTTTTTGTATGTAGCAGATTCAAGTTTGAGTTTTCAATGTGTTATGAATTTAACTAATGATTCTTTTCTAACATGGTGTCTACGTTACTGGCATATTTGGGGCGTTACCGTGCTTTTTGTTCTTTTTTTTATTCATATGGGTCGTGCTTTATATTATTCAAGTTACTCTAAGAAGGGTGTGTGGAACGTTGGGTTTATACTGTATTTGTTAACTATGGCAGAAGCATTCATGGGTTACATTTTACCTTGACACCAAATGTCGTATTGGGCTGCTACTGTTTTAACTTCTATCCTTCAAAGGTTGCCTTTTGTGGGCCCTACTTTATACAAGTATGTGGTCGGTGGGTTTTCTGTGACTAATGACACTTTAGTTCGTGTGTTTTCGGCACATGTGTGTCTTGGTTTTGTTATTATGGGTTTAATGGTACTACATTTATTTTACTTACATAAGACTGGCTCGAAAAATCCTCTTTTTTCTTCGCATGGCTATGGAGACGTAGTTTTTTTTCACTCCTACTTTACTGTGAAGGATTTTTTTTGCTTGGTCTTAGTGACCTTAGTTATGGTCAGGTTCCTACTTTTT------AACCCAGATATGGTGGTTGACGTAGAAGGTTATTTAGAGGCTAATCCATTAGCTACGCCGGTTAGTATAAAGCCTGAGTGGTATTTTTTAGTTTTTTACGCTATGCTTCGATGCATAGAGTCTAAGATAGGAGGGTTAGTCATAATCTTAGCTTTATTATTTTTTTTATGGGTTCCTACTTTTAA----------------CCA-CTCCAGCAGC---

In [38]:
from Bio.Align.Applications import ClustalwCommandline
>>> help(ClustalwCommandline)

Help on class ClustalwCommandline in module Bio.Align.Applications._Clustalw:

class ClustalwCommandline(Bio.Application.AbstractCommandline)
 |  ClustalwCommandline(cmd='clustalw', **kwargs)
 |  
 |  Command line wrapper for clustalw (version one or two).
 |  
 |  http://www.clustal.org/
 |  
 |  Notes
 |  -----
 |  Last checked against versions: 1.83 and 2.1
 |  
 |  References
 |  ----------
 |  Larkin MA, Blackshields G, Brown NP, Chenna R, McGettigan PA,
 |  McWilliam H, Valentin F, Wallace IM, Wilm A, Lopez R, Thompson JD,
 |  Gibson TJ, Higgins DG. (2007). Clustal W and Clustal X version 2.0.
 |  Bioinformatics, 23, 2947-2948.
 |  
 |  Examples
 |  --------
 |  >>> from Bio.Align.Applications import ClustalwCommandline
 |  >>> in_file = "unaligned.fasta"
 |  >>> clustalw_cline = ClustalwCommandline("clustalw2", infile=in_file)
 |  >>> print(clustalw_cline)
 |  clustalw2 -infile=unaligned.fasta
 |  
 |  You would typically run the command line with clustalw_cline() or via
 |  the

In [94]:
from Bio.Align.Applications import ClustalwCommandline
in_file = "missingcytb.fsa"
out_file = "missingcytb_nuc.aln"
clustalw_cline = ClustalwCommandline("clustalo", infile=in_file, outfile=out_file)
print(clustalw_cline)


clustalo -infile=missingcytb.fsa -outfile=missingcytb_nuc.aln


In [95]:
clustalw_cline()

('Using 8 threads\nRead 34 sequences (type: DNA) from missingcytb.fsa\nnot more sequences (34) than cluster-size (100), turn off mBed\nCalculating pairwise ktuple-distances...\nKtuple-distance calculation progress: 0 % (0 out of 595)\nKtuple-distance calculation progress: 6 % (40 out of 595)\nKtuple-distance calculation progress: 17 % (106 out of 595)\nKtuple-distance calculation progress: 21 % (126 out of 595)\nKtuple-distance calculation progress: 22 % (132 out of 595)\nKtuple-distance calculation progress: 32 % (194 out of 595)\nKtuple-distance calculation progress: 34 % (204 out of 595)\nKtuple-distance calculation progress: 36 % (218 out of 595)\nKtuple-distance calculation progress: 38 % (230 out of 595)\nKtuple-distance calculation progress: 39 % (237 out of 595)\nKtuple-distance calculation progress: 40 % (240 out of 595)\nKtuple-distance calculation progress: 43 % (257 out of 595)\nKtuple-distance calculation progress: 48 % (290 out of 595)\nKtuple-distance calculation progres

In [114]:
alignment2 = AlignIO.read(open("missingcytb_nuc.aln"), "clustal")
print("Alignment length %i" % alignment.get_alignment_length())
for record in alignment2:
    print(record.seq + " " + record.id)

Alignment length 1239
---------------ATGACCAACCCACGTAAAACATATCTACTCATAAAAATCATCAACAAGACTTTCATTGACCTTCCAACCCCATCCAACATCTCTACCTGATGAAACTTTGGATCCCTACTAGGCATATGCCTAATAATTCAAATCATTACAGGTCTTTTCTTAGCCATACACTACACCCCAGACACCTCAACTGCCTTCTCATCAGTAGCCCACATCTGCCGAGAAGTAAACTATGGCTGACTTATCCGCAACCTACATGCTAACGGAGCATCAGTCTTCTTCATATGCCTTTATCTCCACATCGGCCGAGGAATTTATTATGGCTCATACCTATATAAAAGCACATGAAATATCGGAGTAATTCTACTACTAATAACTATAATAACTGCTTTCATAGGATATGTACTCCCATGAGGACAAATATCATTTTGAGGGGCGACAGTAATTACCAATCTACTATCCGCCATCCCATACATTGGCAGCACGCTAGTAGAATGAATCTGAGGAGGCTTCTCTGTAGACAAAGCTACTTTAACCCAATTCTTTGCCCTACATTTCATCTTACCATTCATCATTACAGCCCTAACCATAGTCCACCTCTTATTCCTCCACGAAACAGGATCAAACAACCCCTTAGGAATTAACTCTAACCCTGATA---AAATCCCATTTCACCCTTACTATACCACTAAAGACACGATAGGCCTAATACTATTATTAATTT---TACTCAC------ACTAACCCTACTCTCACCTGATACACTAAGTGACCCCAACAACTTTTCCCCTGCAAATCCTCTCAACACACCACAACATATTAAACCAGAATGATGCTTCTTATTTGCATACACAATCCTATGATCCATTCCCAACAAGCTAGGAGGAGTCTTAGCCCTACTAGCATCTATCCTAGTCCTATTTATTATGCCATTCCTCCACACATC---CAAACAAAGCAGCCTAATATTC

In [96]:
from Bio.Align.Applications import ClustalwCommandline
in_file = "missingcytb.pro"
out_file = "missingcytb_pro.aln"
clustalw_cline = ClustalwCommandline("clustalo", infile=in_file, outfile=out_file)
print(clustalw_cline)

clustalo -infile=missingcytb.pro -outfile=missingcytb_pro.aln


In [97]:
clustalw_cline()

('Using 8 threads\nRead 34 sequences (type: Protein) from missingcytb.pro\nnot more sequences (34) than cluster-size (100), turn off mBed\nCalculating pairwise ktuple-distances...\nKtuple-distance calculation progress: 0 % (0 out of 595)\nKtuple-distance calculation progress: 1 % (6 out of 595)\nKtuple-distance calculation progress: 12 % (76 out of 595)\nKtuple-distance calculation progress: 21 % (125 out of 595)\nKtuple-distance calculation progress: 25 % (152 out of 595)\nKtuple-distance calculation progress: 32 % (195 out of 595)\nKtuple-distance calculation progress: 34 % (207 out of 595)\nKtuple-distance calculation progress: 36 % (215 out of 595)\nKtuple-distance calculation progress: 38 % (231 out of 595)\nKtuple-distance calculation progress: 40 % (241 out of 595)\nKtuple-distance calculation progress: 44 % (263 out of 595)\nKtuple-distance calculation progress: 51 % (307 out of 595)\nKtuple-distance calculation progress: 58 % (347 out of 595)\nKtuple-distance calculation progr

In [118]:
pro_alignment = list(AlignIO.read(open("missingcytb_pro.aln"), "clustal"))
print("Alignment length %i" % alignment.get_alignment_length())
for record in pro_alignment:
    print(record.seq + " " + record.id)

Alignment length 1239
-----MTNPRKTYLLMKIINKTFIDLPTPSNISTWWNFGSLLGMCLMIQIITGLFLAMHYTPDTSTAFSSVAHICREVNYGWLIRNLHANGASVFFMCLYLHIGRGIYYGSYLYKSTWNIGVILLLMTMMTAFMGYVLPWGQMSFWGATVITNLLSAIPYIGSTLVEWIWGGFSVDKATLTQFFALHFILPFIITALTMVHLLFLHETGSNNPLGINS-NPDKIPFHPYYTTKDTMGLMLLLI-LLTLTLLS--PDTLSDPNNFSPANPLNTPQHIKPEWCFLFAYTILWSIPNKLGGVLALLASILVLFIMPFLHT-SKQSSLMFRPISQTLFWILVTNLLVLTWIGSQPV-EQSYTTIGQVASTTYFLLIIILMPMAGLLENFMLKPK*----------- NC_008133.1
----MSHRLRLDNPLFKIINGTLIDLPSPVNFSIWWNFGSLLGLVLGIQLLTGIFLAMHYTADVELAFSSVSHIFRDVNSGWLLRSTHANGASFFFICLYSHVGRGLYYGSYMYVKTWITGVLLLILVMAAAFLGYVLPWGQMSFWGATVITNLFSAFPYFGPDLVLWLWGGFAVDNATLTRFFTFHFMVPFIVAAMAAIHIFFLHTTGSNNPLGVNS-DAEKIPFHWYYSIKDVVGFVVMVFFLLSLVLFS--PNYLGEPDNFIAANPMVTPPHIVPEWYFLFAYAILRSIPNKLGGVLGLFGSLLILLSLPVLHI-NSMKGTSYYPFSKFLFWSFVISFFMLTLGGAWPV-EYPYVAVSQFFSVYYFSFFLVSTPLRMYEIL*----------------- NC_013810.1
----MTMRFRLTNKLAKIVNGMVWDLGTPQNINMFWNFGSLLGLMLSIQLITGIFLALQYSANSDLSFFMISRLLRDVNYGWLLRYIHANGASFFFICIYFHIGRGIYYSAYLSKGVWFVGIILLILVMAAAFLGYVLPWGQMSFWGA

In [122]:
type(pro_alignment)

list

In [128]:
alignment3 = AlignIO.read("missingcytb_pro.aln", "clustal")
print(alignment3)

Alignment with 34 rows and 402 columns
-----MTNPRKTYLLMKIINKTFIDLPTPSNISTWWNFGSLLGM...--- NC_008133.1
----MSHRLRLDNPLFKIINGTLIDLPSPVNFSIWWNFGSLLGL...--- NC_013810.1
----MTMRFRLTNKLAKIVNGMVWDLGTPQNINMFWNFGSLLGL...--- NC_013811.1
-MMNLYKPIRKTHPVIKIINGSLIDLPTPSNISSWWNFGSLLAL...--- NC_014587.1
--------MSLSKTGLLEPVNDIIDLPTPSNISYWWNFGSLLCL...--- NC_015309.1
-MMNMFKPMRKTHPIMKIMNNSLIDLPSPSNISSWWNFGSILAL...--- NC_018029.1
-MMNLYKSIRKTHPVIKIINGSLIDLPSPSNISMWWNFGSLLAL...--- NC_018030.1
-MMNKFPPLRKTHPLFKIINNSLIDLPSPSNISAWWNFGSLLAL...--- NC_018047.1
-MMNKFKPLRKTHPLIKILNNSLIDLPSPSNISNLWNFGSLLAL...--- NC_018048.1
-MMNKFLPLRKTHPLFKIINGSLIDLPSPSNISLWWNFGSLLAL...--- NC_018049.1
----MALNLRKNHPLLKIINDSLIDLPTPSNISTWWNFGSLLGI...--- NC_019664.1
----MALNLRKNHPLLKTINNALIDLPTPSNISTWWNFGSLLGI...--- NC_019665.1
----MAPNLRKNHPLLKIINDSLVDLPTPSNISTWWNFGSLLGI...--- NC_019666.1
----MAPNLRKNHPLLKIVNDSLIDLPTPSNISAWWNFGSLLGV...--- NC_019667.1
----MAPNLRKNHPLLKIINDSLVDLPTPSNISTWWNFGSLLGI...--- NC_019668.1
--------------ML

In [132]:
with open('missingcytb_pro.aln', 'w') as file:


<_io.TextIOWrapper name='missingcytb_pro.aln' mode='w' encoding='UTF-8'>


## Score alignments
### https://github.com/benhid/pyMSA/blob/master/resources/tutorial-pymsa.pdf

In [98]:
pip install pyMSA

  Preparing metadata (setup.py) ... done
  Created wheel for pyMSA: filename=pyMSA-0.8.1-py3-none-any.whl size=12824 sha256=a71b85fe50802981c74b4b334beb49f7000a39468af1baa4e08c2d21e2e21e05
  Stored in directory: /Users/rich/Library/Caches/pip/wheels/c7/00/54/1acf56628af0e3684b5ba4395448fa3ca59700ce22994e5935
Successfully built pyMSA
Note: you may need to restart the kernel to use updated packages.


In [133]:
from pymsa import MSA, Entropy, PercentageOfNonGaps, PercentageOfTotallyConservedColumns, Star, SumOfPairs
from pymsa import PAM250, Blosum62, FileMatrix
from pymsa.util.fasta import print_alignment


def run_all_scores(sequences: list) -> None:
    aligned_sequences = list(pair[1] for pair in sequences)
    sequences_id = list(pair[0] for pair in sequences)

    msa = MSA(aligned_sequences, sequences_id)
    print_alignment(msa)

    # Percentage of non-gaps and totally conserved columns
    non_gaps = PercentageOfNonGaps(msa)
    totally_conserved_columns = PercentageOfTotallyConservedColumns(msa)

    percentage = non_gaps.compute()
    print("Percentage of non-gaps: {0} %".format(percentage))

    conserved = totally_conserved_columns.compute()
    print("Percentage of totally conserved columns: {0}".format(conserved))

    # Entropy
    value = Entropy(msa).compute()
    print("Entropy score: {0}".format(value))

    # Sum of pairs
    value = SumOfPairs(msa, Blosum62()).compute()
    print("Sum of Pairs score (Blosum62): {0}".format(value))

    value = SumOfPairs(msa, PAM250()).compute()
    print("Sum of Pairs score (PAM250): {0}".format(value))

    value = SumOfPairs(msa, FileMatrix('PAM380.txt')).compute()
    print("Sum of Pairs score (PAM380): {0}".format(value))

    # Star
    value = Star(msa, Blosum62()).compute()
    print("Star score (Blosum62): {0}".format(value))

    value = Star(msa, PAM250()).compute()
    print("Star score (PAM250): {0}".format(value))


if __name__ == '__main__':
    sequences = [("1g41",
                  "S-EMTPREIVSELDQHIIGQADAKRAVAIALRNRWRRMQLQEPLRHE--------VTP-KNILMIGPTGVGKTEIARRLAKLANAPFIKVEATKFT----"
                  "VGKEVDSIIRDLTDSAMKLVRQQEIAKNR---------------------------------------------------------------------LI"
                  "DDEAAKLINPEELKQKAIDAVE--QNGIVFIDEIDKICKKGEYSGADVSREGVQRDLLPLVEGSTVSTKHGMVKTDHILFIASGAFQVARPSDL------"
                  "-----------IPELQGRLPIR-VEL---TALSAADFERILTEPHASLTEQYKALMATEGVNIAFTTDAVKKIAEAAFRVNEKTENIGARRLHTVMERLM"
                  "DKISFSASDMNGQTVNIDAAYVADALGEVVENEDLSRFIL"),
                 ("1e94",
                  "HSEMTPREIVSELDKHIIGQDNAKRSVAIALRNRWRRMQLNEELRHE--------VTP-KNILMIGPTGVGKTEIARRLAKLANAPFIKVEATKFTEVGY"
                  "VGKEVDSIIRDLTDAAVKMVRVQAIEKNRYRAEELAEERILDVLIPPAKNNWGQTEQQQEPSAARQAFRKKLREGQLDDKEIEKQKARKLKIKDAMKLLI"
                  "EEEAAKLVNPEELKQDAIDAVE--QHGIVFIDEIDKICKRGESSGPDVSREGVQRDLLPLVEGCTVSTKHGMVKTDHILFIASGAFQIAKPSDL------"
                  "-----------IPELQGRLPIR-VEL---QALTTSDFERILTEPNASITVQYKALMATEGVNIEFTDSGIKRIAEAAWQVNESTENIGARRLHTVLERLM"
                  "EEISYDASDLSGQNITIDADYVSKHLDALVADEDLSRFIL"),
                 ("1e32",
                  "R-ED-EEESLNEVGYDDVGG--CRKQLAQ-----I-KEMVELPLRHPALFKAIGVKPP-RGILLYGPPGTGKTLIARAVANETGAFFFLINGPEIM-SKL"
                  "A-GESESN--------------------------------------------------------------------------------------------"
                  "-------------LRKAFEEAEKNAPAIIFIDELDAIAPKREKTHGEVERRIVSQ-LLTLMDGL--------KQRAHVIVMAATN----RPNSIDPALRR"
                  "FGRFDREVDIGIPDATGRLEILQIHTKNMKLADDVDLEQVANETHGH---------------------------------------VGADLAALCSEAAL"
                  "QAIRKKMDLIDLEDETIDAEVM-NSL-AVTMDDFRWALSQ"),
                 ("1d2n",
                  "------EDYASYIMNGIIKWGDP---VTRVLD--DGELLVQQTKNSD--------RTPLVSVLLEGPPHSGKTALAAKIAEESNFPFIKICSPDKM-IGF"
                  "SETAKCQA--------------------------------------------------------------------------------------------"
                  "-------------MKKIFDDAYKSQLSCVVVDDIERLLDYV-PIGPRFSNLVLQA-LLVLLKKA-------PPQGRKLLIIGTTS----R-KDVLQEMEM"
                  "LNA---------------------------------FSTTIHVPNIATGEQL--LEALEL-LGNFKDKE---RTTIAQQVKGKKVWIGIKKLLMLIEM--"
                  "-------------SLQMDPEYRVRKFLALLREEGAS-PLD")]
    run_all_scores(sequences)

1g41	S-EMTPREIVSELDQHIIGQADAKRAVAIALRNRWRRMQLQEPLRHE--------VTP-KNILMIGPTGVGKTEIARRLAKLANAPFIKVEATKFT----
1e94	HSEMTPREIVSELDKHIIGQDNAKRSVAIALRNRWRRMQLNEELRHE--------VTP-KNILMIGPTGVGKTEIARRLAKLANAPFIKVEATKFTEVGY
1e32	R-ED-EEESLNEVGYDDVGG--CRKQLAQ-----I-KEMVELPLRHPALFKAIGVKPP-RGILLYGPPGTGKTLIARAVANETGAFFFLINGPEIM-SKL
1d2n	------EDYASYIMNGIIKWGDP---VTRVLD--DGELLVQQTKNSD--------RTPLVSVLLEGPPHSGKTALAAKIAEESNFPFIKICSPDKM-IGF

1g41	VGKEVDSIIRDLTDSAMKLVRQQEIAKNR---------------------------------------------------------------------LI
1e94	VGKEVDSIIRDLTDAAVKMVRVQAIEKNRYRAEELAEERILDVLIPPAKNNWGQTEQQQEPSAARQAFRKKLREGQLDDKEIEKQKARKLKIKDAMKLLI
1e32	A-GESESN--------------------------------------------------------------------------------------------
1d2n	SETAKCQA--------------------------------------------------------------------------------------------

1g41	DDEAAKLINPEELKQKAIDAVE--QNGIVFIDEIDKICKKGEYSGADVSREGVQRDLLPLVEGSTVSTKHGMVKTDHILFIASGAFQVARPSDL------
1e94	EEEAAKLVNPEELKQDAIDAVE--QHGIVFIDEIDKICK